# 아마존 세이지메이커 처리 작업을 통한 특성 변환

일반적으로 머신러닝(ML) 프로세스는 몇 가지 단계로 구성됩니다. 다양한 ETL 작업을 통해 데이터를 수집 및 전처리하며, 표준 기술이나 사전 지식을 활용하여 데이터 세트를 특징화하고, 마지막으로 알고리즘을 사용하여 ML 모델을 훈련시킵니다.

대부분의 경우 Scikit-Learn, Spark, Ray 등의 분산 데이터 처리 프레임워크를 사용하여 데이터 세트를 전처리하고 훈련을 준비합니다. 이 노트북에서는 아마존 세이지메이커 처리 작업을 사용하고 관리형 세이지메이커 환경에서 허깅페이스의 강력한 기능을 활용하여 처리 작업을 수행합니다.

# 주의: 이 노트북은 완료하는데 5-10분 정도 걸립니다.

# 잠시 기다려 주세요.

## 목차

1. 환경 설정
1. 입력 데이터 설정
1. 출력 데이터 설정
1. 처리 작업을 실행하기 위한 Scikit-Learn 컨테이너 구축
1. 아마존 세이지메이커를 사용하여 처리 작업 실행
1. 처리된 출력 데이터 검사

# 환경 설정

다음 사항을 지정하면서 시작합니다:
* 훈련 및 모델 데이터를 위해 사용할 S3 버킷과 접두사(prefix)를 설정합니다. 아마존 세이지메이커 세션에서 지정한 기본 버킷을 사용합니다.
* 데이터 세트에 대한 처리 및 훈련 액세스를 제공하는데 사용되는 IAM 역할 ARN을 설정합니다.

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='gaia/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)
s3 = boto3.Session().client(service_name="s3", 
                            region_name=region,
                            config=config)

# 입력 데이터 S3 URI 설정

In [ ]:
sess.upload_data('./data-summarization/dialogsum-1.csv', bucket=bucket, key_prefix='data-summarization')
sess.upload_data('./data-summarization/dialogsum-2.csv', bucket=bucket, key_prefix='data-summarization')

In [ ]:
raw_input_data_s3_uri = f's3://{bucket}/data-summarization/'
print(raw_input_data_s3_uri)

In [ ]:
!aws s3 ls $raw_input_data_s3_uri

# 아마존 세이지메이커를 사용하여 처리 작업 실행

아마존 세이지메이커 파이썬 SDK를 사용하여 사용자 정의 파이썬 스크립트를 사용한 처리 작업을 등록합니다.

# 처리 스크립트 검토

In [ ]:
!pygmentize preprocess.py

이 스크립트를 처리 작업으로 실행합니다. 또한 아마존 S3 버킷의 `source`인자와 `/opt/ml/processing/input`(Docker 컨테이너 내부)에서 데이터를 읽어오는 `destination`이 포함된 하나의 `ProcessingInput`을 지정해야 합니다. 처리 컨테이너 내부의 모든 로컬 경로는 `/opt/ml/processing/`로 시작해야 합니다.

또한 `run()` 메서드에 `ProcessingOutput`을 지정합니다. 여기서 `source`는 스크립트가 출력 데이터를 기록하는 경로입니다. 출력의 경우, `destination`은 기본적으로 아마존 세이지메이커 파이썬 SDK가 생성하는 S3 버킷으로 설정되며, 형식은 `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`입니다. 또한 `ProcessingOutput`의 `output_name` 값을 지정하여 작업이 실행된 후에 이러한 출력 아티팩트를 더 쉽게 검색할 수 있습니다.

`run()` 메서드의 인자 매개변수는 `preprocess.py` 스크립트의 명령줄 인자입니다.

데이터를 클러스터의 모든 작업자 노드에 걸쳐 변환을 분산시키기 위해 `ShardedByS3Key`를 사용하여 데이터를 샤딩하는 것에 유의하세요.

In [ ]:
processing_instance_type = "ml.c5.2xlarge"
processing_instance_count = 2
train_split_percentage = 0.9
validation_split_percentage = 0.05
test_split_percentage = 0.05

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=7200,
)

In [ ]:
input_s3 = f's3://{bucket}/data-summarization/'

In [ ]:
!aws s3 ls {input_s3}

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor.run(
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            input_name="raw-input-data",
            source=raw_input_data_s3_uri,
            destination="/opt/ml/processing/input/data/",
            s3_data_distribution_type="ShardedByS3Key",
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train", 
            s3_upload_mode="EndOfJob", 
            source="/opt/ml/processing/output/data/train"
        ),
        ProcessingOutput(
            output_name="validation",
            s3_upload_mode="EndOfJob",
            source="/opt/ml/processing/output/data/validation",
        ),
        ProcessingOutput(
            output_name="test", 
            s3_upload_mode="EndOfJob", 
            source="/opt/ml/processing/output/data/test"
        ),
    ],
    arguments=[
        "--train-split-percentage",
        str(train_split_percentage),
        "--validation-split-percentage",
        str(validation_split_percentage),
        "--test-split-percentage",
        str(test_split_percentage),
        "--model-checkpoint",
        str(model_checkpoint),
    ],
    logs=True,
    wait=False,
)

In [ ]:
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]
print(scikit_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">처리 작업</a> 검토하기</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>약 5분 후 <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch 로그</a> 검토하기</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>처리 작업이 완료된 후 <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 출력 데이터</a> 검토하기</b>'.format(
            bucket, scikit_processing_job_name, region
        )
    )
)

# 처리 작업 모니터링

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

# 위의 ^^ 처리 작업 ^^이 완료될 때까지 기다려 주세요.

# 처리된 출력 데이터 검사

처리 작업이 성공적으로 완료되었는지 확인하기 위해 변환된 데이터 세트의 몇몇 행을 살펴봅니다.

In [ ]:
processing_job_description = running_processor.describe()

output_config = processing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train":
        processed_train_data_s3_uri = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "validation":
        processed_validation_data_s3_uri = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test":
        processed_test_data_s3_uri = output["S3Output"]["S3Uri"]

print(processed_train_data_s3_uri)
print(processed_validation_data_s3_uri)
print(processed_test_data_s3_uri)

In [ ]:
!aws s3 ls $processed_train_data_s3_uri/

In [ ]:
!aws s3 ls $processed_validation_data_s3_uri/

In [ ]:
!aws s3 ls $processed_test_data_s3_uri/

# 다음 노트북으로 변수 전달하기

In [ ]:
%store raw_input_data_s3_uri

In [ ]:
%store train_split_percentage

In [ ]:
%store validation_split_percentage

In [ ]:
%store test_split_percentage

In [ ]:
# %store balance_dataset

In [ ]:
%store processed_train_data_s3_uri

In [ ]:
%store processed_validation_data_s3_uri

In [ ]:
%store processed_test_data_s3_uri

In [ ]:
%store